In [1]:
import subprocess, os, time
import cml.workers_v1 as workers

DASHBOARD_PORT = os.environ['CDSW_READONLY_PORT']
DASHBOARD_IP = os.environ['CDSW_IP_ADDRESS']

# use num-cpus=0 when start a head node to prevent this node from performing task/actor computation.
command = "ray start --head --block --include-dashboard=true --dashboard-port=$CDSW_READONLY_PORT --num-cpus=0 --num-gpus=0 &" 
#command = "ray start --head --block --include-dashboard=true --dashboard-port=$CDSW_READONLY_PORT --num-cpus=6 --num-gpus=1 &" 

subprocess.run(command, shell = True, executable="/bin/bash")

with open("RAY_HEAD_IP", 'w') as output_file:
    output_file.write(DASHBOARD_IP)
            
ray_head_addr = DASHBOARD_IP + ':6379'
ray_url = f"ray://{DASHBOARD_IP}:10001" 
worker_start_cmd = f"!ray start --block --address={ray_head_addr}"

time.sleep(7)
ray_workers = workers.launch_workers(
    n=1, 
    cpu=5, 
    memory=32,
    nvidia_gpu=1,
    code=worker_start_cmd,
)

In [ ]:

    # DeepSpeed correctly handles device placement (CPU or GPU)
    #device = get_accelerator().device_name(model.local_rank)
    device = "cuda:0"

if __name__ == "__main__":
    # --- Conditionally set device based on CUDA availability ---
    use_gpu = torch.cuda.is_available()
    if use_gpu:
        print("✅ CUDA is available. Training will run on GPU.")
    else:
        print("⚠️ CUDA not found. Training will run on CPU.")

    # --- DeepSpeed Configuration ---
    deepspeed_config = {
        "optimizer": {"type": "AdamW", "params": {"lr": 2e-5}},
        "scheduler": {"type": "WarmupLR", "params": {"warmup_num_steps": 100}},
        "fp16": {"enabled": use_gpu}, # Enable fp16 only if GPU is available
        "bf16": {"enabled": False},
        "zero_optimization": {
            "stage": 3,
            "offload_optimizer": {"device": "none"},
            "offload_param": {"device": "none"},
        },
        "gradient_accumulation_steps": 1,
        "gradient_clipping": True,
        "steps_per_print": 10,
        "train_micro_batch_size_per_gpu": 16,
    }


In [2]:
import os
import json
from tempfile import TemporaryDirectory
import torch
import deepspeed
from deepspeed.accelerator import get_accelerator
from torchmetrics.text import ROUGEScore
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, set_seed

import ray
import ray.train
from ray.train import Checkpoint, DataConfig, ScalingConfig
from ray.train.torch import TorchTrainer

# NLTK is required for ROUGE score calculation
import nltk
nltk.download("punkt", quiet=True)


def train_func(config):
    """
    Your training function that will be launched on each Ray worker.
    """
    # Unpack training configs
    set_seed(config["seed"])
    model_id = config["model_id"]
    num_epochs = config["num_epochs"]
    train_batch_size = config["train_batch_size"]
    eval_batch_size = config["eval_batch_size"]
    generation_max_length = config["generation_max_length"]
    generation_num_beams = config["generation_num_beams"]
    deepspeed_config = config["deepspeed_config"]

    # Instantiate the Tokenizer and Model
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

    if model.config.pad_token_id is None:
        model.config.pad_token_id = tokenizer.pad_token_id

    # --- Prepare Ray Data Loaders ---
    train_ds = ray.train.get_dataset_shard("train")
    eval_ds = ray.train.get_dataset_shard("validation")

    def collate_fn(batch):
        # Ensure all data is string type, replacing None with an empty string ""
        instructions = [str(s) if s is not None else "" for s in batch["instruction"]]
        inputs_col = [str(s) if s is not None else "" for s in batch["input"]]
        targets = [str(s) if s is not None else "" for s in batch["output"]]
        
        # Combine instruction and input columns to create the full input prompt
        inputs = [
            f"Instruction: {instr}\nInput: {inp}" 
            for instr, inp in zip(instructions, inputs_col)
        ]

        model_inputs = tokenizer(
            inputs,
            max_length=256,
            padding="longest",
            truncation=True,
            return_tensors="pt"
        )

        labels = tokenizer(
            text_target=targets,
            max_length=generation_max_length,
            padding="longest",
            truncation=True,
            return_tensors="pt"
        ).input_ids

        labels[labels == tokenizer.pad_token_id] = -100
        model_inputs["labels"] = labels

        return model_inputs

    train_dataloader = train_ds.iter_torch_batches(
        batch_size=train_batch_size, collate_fn=collate_fn
    )
    eval_dataloader = eval_ds.iter_torch_batches(
        batch_size=eval_batch_size, collate_fn=collate_fn
    )

    # --- Initialize DeepSpeed Engine ---
    model, optimizer, _, lr_scheduler = deepspeed.initialize(
        model=model,
        model_parameters=model.parameters(),
        config=deepspeed_config,
    )
    
    # Set device as requested
    device = "cuda:0"

    # --- Initialize Evaluation Metrics ---
    rouge_metric = ROUGEScore().to(device)

    # --- Training and Evaluation Loop ---
    for epoch in range(num_epochs):
        model.train()
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            model.backward(loss)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        model.eval()
        for batch in eval_dataloader:
            labels = batch.pop("labels").to(device)
            batch_on_device = {k: v.to(device) for k, v in batch.items()}

            with torch.no_grad():
                generated_ids = model.generate(
                    **batch_on_device,
                    max_length=generation_max_length,
                    num_beams=generation_num_beams,
                )

            decoded_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            labels[labels == -100] = tokenizer.pad_token_id
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            rouge_metric.update(decoded_preds, decoded_labels)

        eval_metrics = rouge_metric.compute()
        rouge_metric.reset()

        if model.global_rank == 0:
            print(f"Epoch {epoch}:")
            for key, value in eval_metrics.items():
                print(f"  {key}: {value.item():.4f}")

        # --- Report checkpoint and metrics to Ray Train ---
        with TemporaryDirectory() as tmpdir:
            model.save_checkpoint(tmpdir)
            if torch.distributed.is_initialized():
                torch.distributed.barrier()

            ray.train.report(
                metrics={k: v.item() for k, v in eval_metrics.items()},
                checkpoint=Checkpoint.from_directory(tmpdir)
            )

if __name__ == "__main__":
    # Check for GPU availability
    use_gpu = torch.cuda.is_available()
    print(f"Using GPU: {use_gpu}")

    deepspeed_config = {
        "optimizer": {"type": "AdamW", "params": {"lr": 2e-5}},
        "scheduler": {"type": "WarmupLR", "params": {"warmup_num_steps": 100}},
        "fp16": {"enabled": use_gpu},
        "bf16": {"enabled": False},
        "zero_optimization": {
            "stage": 3,
            "offload_optimizer": {"device": "none"},
            "offload_param": {"device": "none"},
        },
        "gradient_accumulation_steps": 1,
        "gradient_clipping": True,
        "steps_per_print": 10,
        "train_micro_batch_size_per_gpu": 16,
    }

    training_config = {
        "seed": 42,
        "model_id": "t5-small",
        "num_epochs": 1,
        "train_batch_size": 32,
        "eval_batch_size": 32,
        "generation_max_length": 128,
        "generation_num_beams": 4,
        "deepspeed_config": deepspeed_config,
    }

    deepspeed_config["train_micro_batch_size_per_gpu"] = training_config["train_batch_size"]

    ray_datasets = {
        "train": ray.data.read_parquet("wikisql/data/train-00000-of-00001-36d5d5ed0289390f.parquet"),
        "validation": ray.data.read_parquet("wikisql/data/validation-00000-of-00001-3f1ecb1168a6a037.parquet"),
    }

    trainer = TorchTrainer(
        train_func,
        train_loop_config=training_config,
        scaling_config=ScalingConfig(num_workers=1, use_gpu=True),
        datasets=ray_datasets,
        dataset_config=DataConfig(datasets_to_split=["train", "validation"]),
    )

    result = trainer.fit()

    print("Training finished!")
    if result.best_checkpoints:
        best_checkpoint_path = result.best_checkpoints[0][0].path
        print(f"Best checkpoint saved at: {best_checkpoint_path}")

[2025-07-28 13:15:06,906] [WARNING] [real_accelerator.py:209:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.
[2025-07-28 13:15:06,957] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cpu (auto detect)


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status


[2025-07-28 13:17:01,586] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False
Using GPU: False


2025-07-28 13:17:47,851	INFO worker.py:1747 -- Connecting to existing Ray cluster at address: 10.254.10.56:6379...
2025-07-28 13:17:47,875	INFO worker.py:1918 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8100 


Parquet Files Sample 0:   0%|          | 0.00/1.00 [00:00<?, ? file/s]

...

== Status ==
Current time: 2025-07-28 14:41:45 (running for 01:23:45.50)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/40 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:A100)
Result logdir: /tmp/ray/session_2025-07-28_13-13-11_505390_320/artifacts/2025-07-28_13-17-59/TorchTrainer_2025-07-28_13-17-59/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2025-07-28 14:41:50 (running for 01:23:50.54)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/40 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:A100)
Result logdir: /tmp/ray/session_2025-07-28_13-13-11_505390_320/artifacts/2025-07-28_13-17-59/TorchTrainer_2025-07-28_13-17-59/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2025-07-28 14:41:55 (running for 01:23:55.58)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/40 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:A100)
Result logdir: /tmp/ray/session_2025-07-28_13-13-11_505390_320/artifacts/2025-07-28_

(SplitCoordinator pid=790, ip=10.254.5.187) ✔️  Dataset validation_3_2 execution finished in 592.86 seconds


== Status ==
Current time: 2025-07-28 14:51:40 (running for 01:33:40.20)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/40 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:A100)
Result logdir: /tmp/ray/session_2025-07-28_13-13-11_505390_320/artifacts/2025-07-28_13-17-59/TorchTrainer_2025-07-28_13-17-59/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2025-07-28 14:51:45 (running for 01:33:45.25)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/40 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:A100)
Result logdir: /tmp/ray/session_2025-07-28_13-13-11_505390_320/artifacts/2025-07-28_13-17-59/TorchTrainer_2025-07-28_13-17-59/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2025-07-28 14:51:50 (running for 01:33:50.28)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/40 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:A100)
Result logdir: /tmp/ray/session_2025-07-28_13-13-11_505390_320/artifacts/2025-07-28_

(RayTrainWorker pid=715, ip=10.254.5.187) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/cdsw/ray_results/TorchTrainer_2025-07-28_13-17-59/TorchTrainer_483dd_00000_0_2025-07-28_13-17-59/checkpoint_000002)
2025-07-28 14:53:10,420	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/cdsw/ray_results/TorchTrainer_2025-07-28_13-17-59' in 1.2310s.
2025-07-28 14:53:10,426	INFO tune.py:1041 -- Total run time: 5711.06 seconds (5709.34 seconds for the tuning loop).


== Status ==
Current time: 2025-07-28 14:53:10 (running for 01:35:10.57)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/40 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:A100)
Result logdir: /tmp/ray/session_2025-07-28_13-13-11_505390_320/artifacts/2025-07-28_13-17-59/TorchTrainer_2025-07-28_13-17-59/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)


Training finished!
Best checkpoint saved at: /home/cdsw/ray_results/TorchTrainer_2025-07-28_13-17-59/TorchTrainer_483dd_00000_0_2025-07-28_13-17-59/checkpoint_000000
